In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 25.9 MB/s eta 0:00:00


In [ ]:
# Install Package

!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os
import progressbar
from konlpy.tag import Okt
from bs4 import BeautifulSoup
import time
# Import Library

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
### 이건 원본이야 건드리지말고 복사해서 쓸 것.

# Set GPU environment
device = torch.device("cuda:0")
# Install KoBert
bertmodel, vocab = get_pytorch_kobert_model()

# Set BERT Tokenizer

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

# KoBERT Modeling.

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5, ##Categories. 0 = Very Negative, 1 = Negative, 2 = Neutral, 3 = Positive, 4 = Very Positive.
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
# Set GPU environment
device = torch.device("cuda:0")
# Install KoBert
bertmodel, vocab = get_pytorch_kobert_model()

# Set BERT Tokenizer

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

# KoBERT Modeling.

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5, ##Categories. 0 = Very Negative, 1 = Negative, 2 = Neutral, 3 = Positive, 4 = Very Positive.
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
PATH = '/content/drive/MyDrive/Project/LiveCommerse/OpenAI/KoBERT/'
model = torch.load(PATH + 'KoBERT_GPT-turn.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
model.load_state_dict(torch.load(PATH + 'model_state_dict.pt')) 

<All keys matched successfully>

In [ ]:
main_path = '/content/drive/MyDrive/Project/LiveCommerse/'

In [ ]:
sentiword = main_path + 'KnuSentiLex-master/KnuSentiLex-master/SentiWord_Dict.txt'
dictionary = pd.read_csv(sentiword, sep='\t', header=None, names=['word', 'score'])
okt = Okt()

def analyze_sentiment(text):
    words = okt.pos(text, stem=True, norm=True)
    word_scores = []
    for word, pos in words:
        if pos == 'Adjective' or pos == 'Verb' or pos =='Noun':
            try:
                score = dictionary.loc[dictionary['word'] == word]['score'].values[0]
                word_scores.append(score)
            except:
                continue

    if len(word_scores) > 0:
        avg_score = sum(word_scores) / len(word_scores)
    else:
        avg_score = 0.0

    if avg_score > 0:
        sentiment = 'Positive'
    elif avg_score < 0:
        sentiment = 'Negative'
    else:
        sentiment = 'Neutral'

    return sentiment, avg_score

In [ ]:
Path_Beauty = '/content/drive/MyDrive/Project/LiveCommerse/FUCKUGOOGLE/Beauty/Video'
Path_Food = '/content/drive/MyDrive/Project/LiveCommerse/FUCKUGOOGLE/Food/Video'
Path_Life = '/content/drive/MyDrive/Project/LiveCommerse/FUCKUGOOGLE/Life/Video'
Path_Fashion = '/content/drive/MyDrive/Project/LiveCommerse/FUCKUGOOGLE/Fashion/Video'
Beauty_list = os.listdir(Path_Beauty)
Food_list = os.listdir(Path_Food)
Life_list = os.listdir(Path_Life)
Fashion_list = os.listdir(Path_Fashion)
path_list = [Path_Beauty, Path_Food, Path_Life, Path_Fashion]

In [ ]:
# 모든 동영상 데이터에 넣기.

for path in path_list:
  video_list = os.listdir(path)
  for video in video_list:
    video_path = (path + '/' + video)
    try:
      video_df = pd.read_csv(video_path)
      video_df = video_df[['name', 'chat']]
      video_df = video_df[~video_df['chat'].str.contains('\t')]
      video_df.dropna(inplace=True)
      video_df.reset_index(inplace=True, drop=True)
      video_df.to_csv(video_path, index=False)

      for j in range(len(video_df)):
        video_df.loc[j,'Sentiment'] = analyze_sentiment(video_df['chat'][j])[0]
        video_df.loc[j, 'Score'] = analyze_sentiment(video_df['chat'][j])[1]
      video_df.to_csv(video_path, index=False)
    except:
      print(f'Wrong Path : {video_path}')

In [ ]:
for path in path_list:
  total_df = pd.DataFrame()
  video_list = os.listdir(path)
  for a, video in enumerate(video_list):
    video_num = video.split('_')[1]
    video_num = video_num.split('.')[0]
    video_path = (path + '/' + video)
    try:
      video_df = pd.read_csv(video_path)
      Dict_Score = video_df['Score'].sum() / len(video_df)
      total_df.loc[a, 'Video_Number'] = video_num
      total_df.loc[a, 'Dict_Score_Average'] = Dict_Score

    except:
      total_df.loc[a, 'Video_Number'] = video_num
      total_df.loc[a, 'Dict_Score_Average'] = 0

  total_df.to_excel(path + '/Total_Excel.xlsx', index=False)

In [ ]:
for path in path_list:
  total_df = pd.DataFrame()
  video_list = os.listdir(path)
  for a, video in enumerate(video_list):
    video_num = video.split('_')[1]
    video_num = video_num.split('.')[0]
    video_path = (path + '/' + video)
    try:
      video_df = pd.read_csv(video_path)

In [ ]:
# 학습된 KoBERT 불러온 뒤 예측. ## 원본

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]
    
    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)
        prediction = out.cpu().detach().numpy().argmax()

    return prediction

In [ ]:
# 학습된 KoBERT 불러온 뒤 예측. # 작동안함 그냥 폐기

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]
    
    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)
        prediction = out.cpu().detach().numpy().argmax()

    return prediction

In [ ]:
# Beauty/Video/Video_912024.csv 까지 진행됨. 여기서부터 해야 함.

In [14]:
### 새로운 방법으로 도전. 잘 하는듯.
map_dict = dict({4: 2, 3: 1, 2: 0, 1: -1, 0: -2})
for path in path_list:
  total_df = pd.DataFrame()
  video_list = os.listdir(path)
  for a, video in enumerate(video_list):
    video_num = video.split('_')[1]
    video_num = video_num.split('.')[0]
    video_path = (path + '/' + video)
    try:
      print(f'\n KoBERT Start. \n Video_Path = {video_path}')
      video_df = pd.read_csv(video_path)
      video_df = video_df[['name', 'chat', 'Sentiment', 'Score']]
      video_df.Score = video_df.Score.astype(int)
      video_df.to_csv(video_path, index=False)
      video_df.to_csv('./temp.tsv', index = False, sep = '\t')
      new_test = nlp.data.TSVDataset('./temp.tsv', field_indices=[1,3], num_discard_samples=1)
      test_set = BERTDataset(new_test , 0, 1, tok, max_len, True, False)
      test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=4)

      for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)): 
        token_ids = token_ids.long().to(device) 
        segment_ids = segment_ids.long().to(device) 
        valid_length= valid_length 
        out = model(token_ids, valid_length, segment_ids)
        prediction = out.cpu().detach().numpy().argmax()
        video_df.loc[batch_id, 'KoBERT_Sentiment_Score_2'] = prediction
      video_df['KoBERT_Sentiment_Score_1'] = video_df['KoBERT_Sentiment_Score_2'].map(lambda x: map_dict.get(x))
      video_df.to_csv(video_path, index=False)
      print(f'Done, {video_path}')
    except:
      print(f'\n wrong, {video_path}')


 KoBERT Start. 
 Video_Path = /content/drive/MyDrive/Project/LiveCommerse/FUCKUGOOGLE/Beauty/Video/Video_917670.csv


<ipython-input-14-586a49d3d4f5>:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):


  0%|          | 0/770 [00:00<?, ?it/s]

Done, /content/drive/MyDrive/Project/LiveCommerse/FUCKUGOOGLE/Beauty/Video/Video_917670.csv

 KoBERT Start. 
 Video_Path = /content/drive/MyDrive/Project/LiveCommerse/FUCKUGOOGLE/Beauty/Video/Video_917291.csv


<ipython-input-14-586a49d3d4f5>:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):


...

  0%|          | 0/380 [00:00<?, ?it/s]

Done, /content/drive/MyDrive/Project/LiveCommerse/FUCKUGOOGLE/Fashion/Video/Video_890518.csv

 KoBERT Start. 
 Video_Path = /content/drive/MyDrive/Project/LiveCommerse/FUCKUGOOGLE/Fashion/Video/Video_930136.csv


<ipython-input-14-586a49d3d4f5>:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):


  0%|          | 0/343 [00:00<?, ?it/s]

Done, /content/drive/MyDrive/Project/LiveCommerse/FUCKUGOOGLE/Fashion/Video/Video_930136.csv


In [ ]:
### 원본

map_dict = dict({4: 2, 3: 1, 2: 0, 1: -1, 0: -2})
for path in path_list:
  total_df = pd.DataFrame()
  video_list = os.listdir(path)
  for a, video in enumerate(video_list):
    video_num = video.split('_')[1]
    video_num = video_num.split('.')[0]
    video_path = (path + '/' + video)
    try:
      video_df = pd.read_csv(video_path)
      bar = progressbar.ProgressBar(maxval = len(video_df)).start()
      for i in range(len(video_df)):
        bar.update(i)
        video_df.loc[i, 'KoBERT_Sentiment_2'] = predict(video_df.loc[i, 'chat'])
        video_df['KoBERT_Sentiment_1'] = video_df['KoBERT_Sentiment_2'].map(lambda x: map_dict.get(x))
        video_df.to_csv(video_path)
      print(f'Done, {video_path}')
      bar.finish()
    except:
      print(f'wrong, {video_path}')

In [ ]:
## 이거 수정

okt = Okt()
for path in path_list[2:4]:
  total_df = pd.DataFrame()
  video_list = os.listdir(path)
  excel_path = path.split('/Video')[0]

  for a, video in enumerate(video_list):
    video_num = video.split('_')[1]
    video_num = video_num.split('.')[0]
    video_path = (path + '/' + video)
    try:
      print(f'start :{video_path}')
      
      video_df = pd.read_csv(video_path)
      temp_list = []
      bar = progressbar.ProgressBar(maxval = len(video_df)).start()
      for j in range(len(video_df)):
        bar.update(j)
        words = okt.pos(video_df['chat'][j], stem=True, norm=True)

        for word, pos in words:
              if pos == 'Adjective' or pos == 'Verb' or pos =='Noun':
                temp_list.append(word)
                  
      temp_list = [word for word in temp_list if len(word) > 1]
      temp_words = '\t'.join(temp_list)

      Dict_Score = video_df['Score'].sum() / len(video_df)
      KoBERT_Score = video_df['KoBERT_Sentiment_Score_1'].sum() / len(video_df)

      pattern = '\d{16,}'
      hello_pattern = '안녕|방가|반가|하이|반갑'
      question_pattern = '요\?|죠\?|용\?|까\?|여\?|염\?|죵\?|용\?'

      drop_df = video_df[~(video_df['chat'].str.contains(pattern))]
      drop_df.reset_index(drop=True, inplace=True)

      len_chat = len(video_df)
      drop_chat = len(drop_df)
      drop_user = drop_df.name.nunique()
      drop_positive_dict = len(drop_df[drop_df['Score']>0])
      drop_negative_dict = len(drop_df[drop_df['Score']<0])
      drop_neutral_dict = len(drop_df[drop_df['Score']==0])
      drop_positive_KoBERT = len(drop_df[drop_df['KoBERT_Sentiment_Score_1']>0])
      drop_negative_KoBERT = len(drop_df[drop_df['KoBERT_Sentiment_Score_1']<0])
      drop_neutral_KoBERT = len(drop_df[drop_df['KoBERT_Sentiment_Score_1']==0])
      drop_hello = len(drop_df[(drop_df['chat'].str.contains(hello_pattern))])
      drop_question = len(drop_df[(drop_df['chat'].str.contains(question_pattern))])

      total_df.loc[a, 'Video_Number'] = video_num
      total_df.loc[a, 'Dict_Score_Average'] = Dict_Score
      total_df.loc[a, 'KoBERT_Score_Average'] = KoBERT_Score
      total_df.loc[a, 'Channel_Event_Chat'] = (len_chat) - (drop_chat)
      total_df.loc[a, 'Channel_No_Event_Chat'] = drop_chat
      total_df.loc[a, 'Channel_Chat_User'] = drop_user
      total_df.loc[a, 'Channel_Chat_Positive_Dict'] = drop_positive_dict
      total_df.loc[a, 'Channel_Chat_Negative_Dict'] = drop_negative_dict
      total_df.loc[a, 'Channel_Chat_Neutral_Dict'] = drop_neutral_dict
      total_df.loc[a, 'Channel_Chat_Positive_KoBERT'] = drop_positive_KoBERT
      total_df.loc[a, 'Channel_Chat_Negative_KoBERT'] = drop_negative_KoBERT
      total_df.loc[a, 'Channel_Chat_Neutral_KoBERT'] = drop_neutral_KoBERT
      total_df.loc[a, 'Channel_Chat_Hello'] = drop_hello
      total_df.loc[a, 'Channel_Chat_Question'] = drop_question
      total_df.loc[a, 'words'] = temp_words

      bar.finish()

    except:
      total_df.loc[a, 'Video_Number'] = video_num
      print(f'Error Occur {video_num}')
  total_df.to_excel(excel_path + '/Total_Excel.xlsx', index=False)